In [15]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-14'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [17]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])

In [ ]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [18]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


KeyError: 'Date'

In [20]:
drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,0,0,36814000.0,37588000.0,36811000.0,644.746527
1,0,0,37310000.0,38174000.0,37257000.0,925.681724
2,0,0,38169000.0,38349000.0,38135000.0,141.149411
3,0,0,38270000.0,38320000.0,38104000.0,163.332002
4,0,0,38273000.0,38298000.0,38023000.0,176.189348


In [21]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
period = 14 
au = ups.abs().ewm(com = period-1, min_periods = period).mean() 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)
df['RSI'] = RSI

0      34.867045
1      39.069299
2      48.462128
3      49.768476
4      49.805505
         ...    
415          NaN
416          NaN
417          NaN
418          NaN
419          NaN
Name: tradePrice, Length: 420, dtype: float64


In [22]:
# CCI 오예
N = 20
D = 0
m = 0
for i in range(3):
    D = 0
    m = 0
    sm= 0

    M = ((df['tradePrice'][i] + df['lowPrice'][i] + df['highPrice'][i]) / 3)

    for j in range(N):
        jM = (df['tradePrice'][i+j] + df['lowPrice'][i+j] + df['highPrice'][i+j]) / 3
        sm +=jM
    sm = (sm/N)

    for k in range(N):
        kM = (df['tradePrice'][i+k] + df['lowPrice'][i+k] + df['highPrice'][i+k]) / 3
        D += abs(sm-kM)
    D = D/N
    print('D : {}  // M : {} // m : {}'.format(D,M,sm))

    CCI = (M - sm) / (D * 0.015)

    print(CCI)


D : 449483.33333333506  // M : 37071000.0 // m : 37993083.33333333
-136.76196126416625
D : 449576.6666666705  // M : 37580333.333333336 // m : 37993016.66666666
-61.19584102575273
D : 447780.00000000297  // M : 38217666.666666664 // m : 37994299.99999999
33.25541808725567


In [33]:
# MACD
period = 14
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
# s_data, l_data = df['tradePrice'].diff() ,df['tradePrice'].diff()
au = s_data.abs().ewm(com = period-1, min_periods = period).mean() 
au

419             NaN
418             NaN
417             NaN
416             NaN
415             NaN
           ...     
4      3.828868e+07
3      3.828735e+07
2      3.827890e+07
1      3.820969e+07
0      3.811000e+07
Name: tradePrice, Length: 420, dtype: float64